# Inteligência Computacional

## Projeto DATA MINING CUP Competition 2013

In [34]:
import pandas as pd

In [35]:
data = pd.read_csv('transact_train.csv')

In [36]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.00,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.94,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39887.00,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15633.00,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429008,49998,18,7,5988882.00,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,n
429009,49999,18,7,675114.00,6,59.0,199.99,509.96,1,89.99,...,?,25038,?,?,?,0,24,1,4,n
429010,49999,18,7,715341.00,7,59.0,649.99,1159.95,1,89.99,...,completely orderable,25038,?,?,?,0,24,1,4,n
429011,50000,18,7,0.00,1,6.99,6.99,6.99,1,6.99,...,?,47,300,537,5,0,53,1,34,n


In [37]:
countLog  = data.groupby(['sessionNo'], as_index=False).count().iloc[:,1]

In [38]:
countLog # Variavel com o numero de aparição de cada sessionNo no log.

0         3
1         5
2         5
3         2
4         8
         ..
49995    31
49996     2
49997     3
49998     2
49999     2
Name: startHour, Length: 50000, dtype: int64

In [81]:
count = 0
active_session = 1
bStep_count = []
print(len(data))
for i in range(len(data)):
    if(int(data['sessionNo'].iloc[i]) == active_session):
        if(data['bStep'].iloc[i]!= "?" and int(data['bStep'].iloc[i]) >= 3):
            count += 1
    else:
        active_session += 1
        bStep_count.append(count)
        count = 0

429013


In [82]:
bStep_count

[0,
 2,
 1,
 0,
 2,
 0,
 2,
 0,
 3,
 0,
 10,
 9,
 3,
 0,
 0,
 5,
 5,
 0,
 0,
 5,
 3,
 0,
 0,
 1,
 0,
 2,
 0,
 3,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 5,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 2,
 10,
 0,
 0,
 4,
 3,
 0,
 3,
 4,
 1,
 0,
 4,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 13,
 3,
 6,
 0,
 0,
 5,
 4,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 5,
 3,
 0,
 0,
 15,
 2,
 0,
 0,
 5,
 0,
 0,
 19,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 4,
 3,
 2,
 0,
 10,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 1,
 1,
 5,
 0,
 0,
 0,
 0,
 2,
 1,
 7,
 0,
 0,
 1,
 0,
 0,
 0,
 5,
 3,
 3,
 0,
 0,
 3,
 3,
 5,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 6,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0

In [83]:
data = data.groupby(['sessionNo'], as_index=False).apply()mode()

AttributeError: Cannot access callable attribute 'mode' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [25]:
data = data.groupby(['sessionNo'], as_index=False).last()

In [26]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,39887.0,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
1,2,6,5,94469.0,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
2,3,6,5,341613.0,11,9.99,29.99,109.95,2,9.99,...,completely orderable,3,1800,475,302,12,45,1,11,y
3,4,6,5,42812.0,4,4.99,4.99,19.96,1,4.99,...,completely not orderable,?,?,?,?,?,?,?,?,n
4,5,6,5,2816046.0,45,12.99,179.95,1093.72,4,19.99,...,completely orderable,4,800,503,18,1,46,1,40,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,18,7,7271812.0,197,6.99,59.99,4315.03,13,9.99,...,completely orderable,?,?,?,?,?,?,?,?,y
49996,49997,18,7,667219.0,8,70.99,89.99,667.88,1,89.99,...,completely orderable,?,?,?,?,?,?,?,?,n
49997,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,n
49998,49999,18,7,715341.0,7,59.0,649.99,1159.95,1,89.99,...,completely orderable,25038,?,?,?,0,24,1,4,n


In [27]:
data.groupby(['availability'], as_index=False).last()


,availability,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,...,onlineStatus,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,?,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,...,?,?,?,?,?,?,?,?,?,n
1,completely not determinable,49533,18,7,4305367.0,40,5.99,69.95,943.45,0,...,n,?,?,?,?,?,?,?,?,n
2,completely not orderable,49974,18,7,1832336.0,42,10.0,79.95,630.62,0,...,y,25023,2700,503,223,19,56,1,35,n
3,completely orderable,50000,18,7,82943.0,1,6.99,6.99,6.99,1,...,y,47,300,537,5,0,53,1,34,n
4,mainly not determinable,39468,10,7,874092.0,11,9.99,29.99,244.77,4,...,n,?,?,?,?,?,?,?,?,n
5,mainly not orderable,48329,17,7,1808881.0,106,1.0,24.99,411.92,3,...,y,24151,600,510,84,24,35,2,37,n
6,mainly orderable,49992,18,7,389549.0,16,5.99,9.99,127.84,4,...,y,?,?,?,?,?,?,?,?,n
7,mixed,49823,18,7,662571.0,7,1039.99,1299.99,7019.94,2,...,y,?,?,?,?,?,?,?,?,n


In [28]:

data.loc[(data.availability == 'completely not determinable'),'availability']= 0
data.loc[(data.availability == 'completely not orderable'),'availability']= 1
data.loc[(data.availability == 'completely orderable'),'availability']= 2
data.loc[(data.availability == 'mainly not determinable'),'availability']= 3
data.loc[(data.availability == 'mainly not orderable'),'availability']= 4
data.loc[(data.availability == 'mainly orderable'),'availability']= 5
data.loc[(data.availability == 'mixed'),'availability']= 6

In [29]:
data.loc[(data.onlineStatus == 'y'),'onlineStatus']= 1
data.loc[(data.onlineStatus == 'n'),'onlineStatus']= 0

In [30]:
data.loc[(data.order == 'y'),'order']= 1
data.loc[(data.order == 'n'),'order']= 0

In [31]:
data

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,39887.0,1,59.99,59.99,59.99,1,59.99,...,2,1,600,70,21,1,43,1,49,1
1,2,6,5,94469.0,0,?,?,?,0,?,...,2,?,?,?,?,?,?,?,?,1
2,3,6,5,341613.0,11,9.99,29.99,109.95,2,9.99,...,2,3,1800,475,302,12,45,1,11,1
3,4,6,5,42812.0,4,4.99,4.99,19.96,1,4.99,...,1,?,?,?,?,?,?,?,?,0
4,5,6,5,2816046.0,45,12.99,179.95,1093.72,4,19.99,...,2,4,800,503,18,1,46,1,40,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,18,7,7271812.0,197,6.99,59.99,4315.03,13,9.99,...,2,?,?,?,?,?,?,?,?,1
49996,49997,18,7,667219.0,8,70.99,89.99,667.88,1,89.99,...,2,?,?,?,?,?,?,?,?,0
49997,49998,18,7,5988882.0,77,9.99,149.99,5253.28,3,49.95,...,?,?,?,?,?,?,?,?,?,0
49998,49999,18,7,715341.0,7,59.0,649.99,1159.95,1,89.99,...,2,25038,?,?,?,0,24,1,4,0
